# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

## Vignesh Madharapakkam Pagadala

## Summary


# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
	* [Functions Implemented](#Functions-Implemented)
	* [Negamax](#Negamax)
        * [Overview](#Overview)
        * [Algorithm Definition](#Algorithm-Definition)
        * [Implementation](#Implementation)
        * [Tic-Tac-Toe](#Tic-Tac-Toe)
        <br><br>
	* [Negamax with Iterative Deepening](#Negamax-with-Iterative-Deepening)
	* [Negamax with Iterative Deepening and Alpha-Beta Pruning](#Negamax-with-Iterative-Deepening-and-Alpha--Beta-Pruning)
	* [Comparision](#Comparision)

### negamax

In [5]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

### classTTT 

In [51]:
class TTT(object):

    def __init__(self):
        self.movesExplored = 0
        self.board = [' ']*9
        self.player = 'X'
        if False:
            #self.board = ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']
            self.board = ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']
            self.player = 'X'
        self.playerLookAHead = self.player

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.movesExplored += 1
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s
    def getNumberMovesExplored(self):
        return self.movesExplored
    def getWinningValue(self):
        return 1

Check that the following function `playGame` runs
correctly. Notice that we are using *negamax* to find the best move for
Player X, but Player O, the opponent, is using function *opponent*
that follows the silly strategy of playing in the first open position.

In [49]:
def opponent(board):
    return board.index(' ')

def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
    print('Number of moves explored', game.getNumberMovesExplored())

In [52]:
game = TTT()
playGame(game,opponent,20)

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 
Number of moves explored 558334


## Add moves counter

Evaluate the efficiency of the search by keepting track of the number of nodes explored, which is the same as the number of moves explored, during a game. Do this by adding a counter named `movesExplored` to the `TTT` constructor where it is initialized to 0 and increment the counter in the `TTT.makeMove` method.  Add a method `ttt.getNumberMovesExplored()` to get its current value.  So

    print('Number of moves explored', game.getMovesExplored())
    
will print the number of moves explored. You will not use a global variable for counting this time.

## negamaxIDS 

<font color='red'>UPDATED Oct 4</font>

Write a new function named `negamaxIDS` that performs an iterative deepening negamax search.  Replace the first line in the `while` loop of `playGame` with

        score,move = negamaxIDS(game,depthLimit)
        
where `depthLimit` is now the maximum depth and multiple `negamax` searches are performed for depth limits of $1, 2, \ldots,$ maximum depth.

But, when should you stop?  Can you stop before readhing the depthLimit?  If not, there is no point to doing iterative deepening.

For Tic-Tac-Toe, we can stop as soon as a call to `negamax` returns a winning move.  This will have a value of 1 for Tic-Tac-Toe.  To keep our `negamaxIDS` function general, add a method called `getWinningValue` to the `TTT` class that just returns 1.  Then `negamaxIDS` can call `game.getWinningValue()` to determine the value of a winning move for this game.  If the maximum depth is reached and no winning move has been found, return the best move found over all depth limts.

In [9]:
def negamaxIDS(game, maxDepth):
    for depth in range(maxDepth + 1):
        # CALLING negamax FOR EACH DEPTH VALUE
        result, move = negamax(game, depth)
        winVal = game.getWinningValue()
        if result == winVal:
            break
    return result, move

In [10]:
game = TTT()
negamaxIDS(game,5)

(1, 0)

Now for the hardest part.  Make a new function `negamaxIDSab` by duplicating `negamaxIDS` and add the code to implement alpha-beta pruning.

## negamaxab - negamax with alpha beta pruning

In [11]:
def negamaxab(game, depthLeft, alpha = -float('inf'), beta = float('inf')):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None

    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamaxab(game, depthLeft-1, -beta, -alpha)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        alpha = max(bestValue, alpha)
        if bestValue >= beta:
            return bestValue, bestMove
    return bestValue, bestMove



def negamaxIDSab(game, maxDepth):
    for depth in range(maxDepth + 1):
        #print("We're at depth: ", depth)
        # CALLING negamax FOR EACH DEPTH VALUE
        result, move = negamaxab(game, depth)
        winVal = game.getWinningValue()
        if result == winVal:
            return result, move
    return result, move

## EBF

def ebf(nNodes, depth, precision=0.01):
    if nNodes == 0:
        return 0
    
    def ebfRec(low, high):
        mid = (low + high) * 0.5
        if mid == 1:
            estimate = 1 + depth
        else:
            estimate = (1 - mid**(depth + 1)) / (1 - mid)
        if abs(estimate - nNodes) < precision:
            return mid
        if estimate > nNodes:
            return ebfRec(low, mid)
        else:
            return ebfRec(mid, high)

    return ebfRec(1, nNodes)

In [12]:
# This function computes the Effective Branching Factor (EBF) for a tree with 'nNodes' nodes and depth 'depth'. 
# We use Binary Search to compute the EBF.

def ebf(nNodes, depth, precision = 0.01):
    
    # Initialize low and high values.
    left = 1
    right = nNodes
    
    # If no nodes in tree return 0
    if nNodes == 0:
        return 0
    
    # Keep iterating till the difference between left and right is lesser than the specified precision.
    b = 1.0
    while(True):
        
        # If the difference is lesser than the precision, then break out.
        if abs(right - left) <= precision:
            break
            
        # Get the middle value
        b = (left + right)/2
        
        # Solve equation to find n
        n = (pow(b, (depth+1)) - 1)/(b - 1)
        if nNodes < n:
            right = b - precision
        elif nNodes > n:
            left = b + precision
        else:
            return b
    return b    

## playGames

Now duplicate the game playing loop so three complete tic-tac-toe games are played.  Call this new version `playGames`. For the first game, use `negamax`. For the second game, use `negamaxIDS`.  For the third game, use `negamaxIDSab`.  At the end of each game, print the number of X's in the final board, the number moves explored, the depth of the game which is the number of moves made by X and O, and the effective branching factor.  When you run `playGames` you should see the tic-tac-toe positions after each move and, after all games are done, a line for each game like the following lines, which were <font color='red'>UPDATED Oct 5</font>.

    negamax made 4 moves. 558334 moves explored for ebf(558334, 7) of 6.46
    negamaxIDS made 3 moves. 744695 moves explored for ebf(744695, 5) of 14.73
    negamaxIDSab made 3 moves 20804 moves explored for ebf(20804, 5) of 7.09

Your results may be different. 

The value of the depth is the total number of moves made by X and by O during the search.  You can calculate this by keeping a list of all board states, or by just counting the number of X's and O's in the final board.

Here are some example results. <font color='red'>Updated October 6, 3:15pm </font>

In [67]:
def playGames(opponent,depthLimit, funlist):
    # Create an empty list to maintain a list of strings which are to be printed at the ned.
    strlst = []
    # Create a list to store scores
    scores = []
    for fun in funlist:
        print()
        print(fun.__name__, ':')
        game = TTT()
        print(game)
        while not game.isOver():
            score,move = fun(game,depthLimit)
            if move == None :
                print('move is None. Stopping.')
                break
            game.makeMove(move)
            print('Player', game.player, 'to', move, 'for score' ,score)
            print(game)
            if not game.isOver():
                game.changePlayer()
                opponentMove = opponent(game.board)
                #print(opponentMove)
                game.makeMove(opponentMove)
                print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
                print(game)
                game.changePlayer()
        # Get depth
        xl = len(game.locations('X'))
        ol = len(game.locations('O'))
        dep =  xl + ol
        strlst.append(fun.__name__ + ' made ' + str(len(game.locations('X'))) + ' moves. ' + str(game.getNumberMovesExplored()) + ' moves explored for ebf(' + str(game.getNumberMovesExplored()) + ', ' + str(dep) + ') of ' + str(round(ebf(game.getNumberMovesExplored(), dep), 2)))
        scores.append(score)
    for s in strlst:
        print(s)
    return scores

In [69]:
playGames(opponent, 10, [negamaxIDSab])


negamaxIDSab :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
negamaxIDSab made 3 moves. 6053 moves explored for ebf(6053, 5) of 5.49


[1]

## Grading

As always, download and extract from [A4grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A4grader.tar)

In [ ]:
# %load A4mysolution.py

In [172]:
%run -i A4grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 8
X| | 
-----
 | | 
-----
 | |O
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 | |O
Player O to 7
X| |X
-----
 | | 
-----
 |O|O
Player X to 1 for 

## Extra Credit

Earn one extra credit point for each of the following.

  - Implement another game and repeat the above steps.

  - Implement a random move chooser as the opponent (Player O) and determine how many times Player X can win against this opponent as an average over multiple games.

In [91]:
class New(object):
    def __init__(self):
        self.movesExplored = 0
        self.board = 0
        self.player = 'A'
        self.playerLookAHead = self.player
        self.acount = 0
        self.bcount = 0 

    def getMoves(self):
        moves = range(1,10)
        moves = list(moves)
        for i in moves:
            n = self.board + i
            if n > 50:
                moves.remove(i)
        return moves

    def getUtility(self):
        if self.board == 50:
            # A wins
            if self.playerLookAHead == 'A':
                return 1
            # B wins
            else:
                return -1
        else:
            return None 

    def isOver(self):
        return self.getUtility() is not None

    # 'move' is a number between 0 and 9
    def makeMove(self, move):
        self.movesExplored += 1
        self.board += move
        
        self.playerLookAHead = 'A' if self.playerLookAHead == 'B' else 'B'

    def changePlayer(self):
        self.player = 'A' if self.player == 'B' else 'B'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board -= move
        self.playerLookAHead = 'A' if self.playerLookAHead == 'B' else 'B'

    def __str__(self):
        return str(self.board)
    def getNumberMovesExplored(self):
        return self.movesExplored
    def getWinningValue(self):
        return 1

    def incCount(self):
        if self.playerLookAHead == 'A':
            self.acount += 1
        else:
            self.bcount += 1

In [92]:
def playGames2(opponent, depthLimit, funlist):
    # Create an empty list to maintain a list of strings which are to be printed at the ned.
    strlst = []
    for fun in funlist:
        print()
        print(fun.__name__, ':')
        game = New()
        print(game)
        while not game.isOver():
            score,move = fun(game,depthLimit)
            if move == None :
                print('move is None. Stopping.')
                break
            game.incCount()
            game.makeMove(move)
            print('Player', game.player, 'to', move, 'for score' ,score)
            print(game)
            if not game.isOver():
                game.changePlayer()
                opponentMove = opponent(game.board)
                #print(opponentMove)
                game.makeMove(opponentMove)
                print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
                print(game)
                game.changePlayer()
        # Get depth
        dep =  game.acount + game.bcount
        strlst.append(fun.__name__ + ' made ' + str(game.acount) + ' moves. ' + str(game.getNumberMovesExplored()) + ' moves explored for ebf(' + str(game.getNumberMovesExplored()) + ', ' + str(dep) + ') of ' + str(round(ebf(game.getNumberMovesExplored(), dep), 2)))
    for s in strlst:
        print(s)

In [93]:
import random as rd
def opponent2(board):
    diff = 50 - board
    if diff <= 8:
        r = rd.randint(1,diff)
    else:
        r = rd.randint(1, 9)
    return r

In [94]:
playGames2(opponent2, 6, [negamaxIDSab])


negamaxIDSab :
0
Player A to 5 for score 1
5
Player B to 7
12
Player A to 2 for score 1
14
Player B to 7
21
Player A to 2 for score 1
23
Player B to 1
24
Player A to 8 for score 1
32
Player B to 3
35
Player A to 6 for score 1
41
Player B to 3
44
Player A to 6 for score 1
50
negamaxIDSab made 6 moves. 657323 moves explored for ebf(657323, 6) of 9.16


## Random move chooser for TTT

In [34]:
def opponent3(board):
    moves = [i for i, mark in enumerate(board) if mark == ' ']
    # Random integer
    r = rd.randint(0, len(moves)-1)
    return moves[r]

In [66]:
playGames(opponent3, 10, [negamaxIDSab])


negamaxIDSab :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 5
X| | 
-----
 | |O
-----
 | | 
Player X to 1 for score 1
X|X| 
-----
 | |O
-----
 | | 
Player O to 7
X|X| 
-----
 | |O
-----
 |O| 
Player X to 2 for score 1
X|X|X
-----
 | |O
-----
 |O| 
negamaxIDSab made 3 moves. 5989 moves explored for ebf(5989, 5) of 5.49


## Playing many times with this 'random' opponent

In [89]:
def newfunc(opponent, depthLimit, funlist, n):
    for fun in funlist:
        print()
        print('Now we trying out function: ', fun.__name__)
        scorelist = []
        for i in range(n):
            #print(i)
            game = TTT()
            while not game.isOver():
                score,move = fun(game,depthLimit)
                if move == None :
                    print('Insufficient depth specified. Stopping game!')
                    break
                game.makeMove(move)
                if not game.isOver():
                    game.changePlayer()
                    opponentMove = opponent(game.board)
                    game.makeMove(opponentMove)
                    game.changePlayer()
            scorelist.append(score)
            #if score != 1:
                #print('sadsad')
                #break
        average = (scorelist.count(1)/len(scorelist)) * 100
        print('In', n, ' games, player X has won', average, "% of the times against player Y." )


In [90]:
newfunc(opponent3, 10, [negamaxIDSab], 50)


Now we trying out function:  negamaxIDSab
In 50  games, player X has won 96.0 % of the times against player Y.
